In [1]:
import warnings

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr
import talib

In [3]:
MONTH = 21
YEAR = 12 * MONTH

In [4]:
START = '2013-01-01'
END = '2017-12-31'

sns.set_style('whitegrid')
idx = pd.IndexSlice

In [8]:
#loading wiki stock prices & meta data
ohlcv = pd.read_csv(r'C:\Users\Wendel\OneDrive\Machine-Learning-for-Algorithmic-Trading-Second-Edition\data\wiki_prices\wiki_prices.csv')[{'date','adj_open','adj_high', 'adj_low','adj_close','adj_volume'}]
ohlcv.set_index('date')

,adj_low,adj_open,adj_high,adj_close,adj_volume
0,27.289627,31.041951,34.112034,30.018590,44739900.0
1,27.160002,29.295415,29.336350,27.548879,10897100.0
2,27.330562,28.183363,30.018590,30.018590,4705200.0
3,27.460188,28.995229,29.766161,27.460188,4274400.0
4,27.289627,27.378319,28.613174,28.012803,3464400.0
...,...,...,...,...,...
15389309,23.605800,23.800000,24.600000,23.950000,354092.0
15389310,23.300000,23.900000,24.350000,23.350000,269607.0
15389311,23.450000,23.550000,24.200000,23.550000,301584.0
15389312,23.700000,23.750000,24.800000,24.650000,375320.0


In [ ]:
min_obs = 